# Forecasting Product Sales

In this notebook we will be using the Northwind Trading database to create a model to forecast sales data. This will allow us to provide accurate predictions for profits and inventory control.

In [1]:
# Modules
import pandas as pd
import psycopg2
import matplotlib.pyplot as plt
import os
from sqlalchemy import create_engine
%matplotlib inline

In [12]:
# Connection info for code readability
conn_info = {
    'dbname': 'northwind',
    'user': 'postgres',
    'password': os.getenv('DB_PASSWORD'),
    'host': 'localhost',
    'port': '5432'
}

## Transforming the data

First we need to pull from the database the needed data. We will need order details such as:
- Product Id
- Product Name (not part of the model but will be used for visuals later)
- Order Date
- Units sold

As the database has inconsistency with the price of units over time changes, we will only apply the monetary values after we have constructed a forecasting model based on product movement, than apply the most up to date pricing. This will also enable the model to show how future profits will change as we change prices of certain items.

Now lets pull the needed data from the database.

In [ ]:
query = 
"""

"""